# JuSpyce API test: data import

In [1]:
import sys
import os
from glob import glob
import pathlib
import numpy as np
import pandas as pd
from IPython.display import display
import seaborn as sns

# current path
wd = pathlib.Path().resolve().parent
print(wd)

# import juspyce
sys.path.append(os.path.dirname(os.path.join(wd, "juspyce")))
from juspyce.api import JuSpyce
from juspyce.stats import *
from juspyce.utils import *

/Users/llotter/projects/juspyce


## Collect data

In [2]:
# imaging data
predictors_vol = glob(os.path.join(wd, "testing", "test_predictors", "*.nii.gz"))
predictors_surf = [
    (os.path.join(wd, "testing", "test_predictors", "CMRo2-o2-33-Vaishnavi2010_L.func.gii"),
     os.path.join(wd, "testing", "test_predictors", "CMRo2-o2-33-Vaishnavi2010_R.func.gii")),
    (os.path.join(wd, "testing", "test_predictors", "CBF-h2o-33-Vaishnavi2010_L.func.gii"),
     os.path.join(wd, "testing", "test_predictors", "CBF-h2o-33-Vaishnavi2010_R.func.gii"))]
functional_vol = glob(os.path.join(wd, "testing", "test_functional", "*.nii.gz"))

# parcellations
parcellation_vol = os.path.join(wd, "testing", "test_parc", "Schaefer100-7_TianS1_2mm.nii.gz")
parcellation_vol_labels = pd.read_csv(
    os.path.join(wd, "testing", "test_parc", "Schaefer100-7_TianS1_2mm.csv")).label.to_list()
parcellation_surf = (os.path.join(wd, "testing", "test_parc", "destrieux.fsa.10k.L.label.gii"),
                     os.path.join(wd, "testing", "test_parc", "destrieux.fsa.10k.R.label.gii"))

print(len(predictors_vol))
print(len(predictors_surf))
print(len(functional_vol))
print(len(parcellation_vol_labels))

7
2
28
116


## Initialize & import: volumetric data

In [3]:
# initialize
juspyce_vol = JuSpyce(
    x=predictors_vol, # list of input data to predict functional data from (e.g., PET volumes)
    y=functional_vol, # list of functional data
    z="gm", # optional: e.g., to partial out in partial correlations, "gm"=use standard gmv volume
    data_space="MNI152", # data space, should be "MNI152" in this case, can be "fsaverage", "fslr"
    parcellation=parcellation_vol,
    parcellation_labels=parcellation_vol_labels,
    parcellation_space="MNI152",
    standardize=True, # will z-transform all input data. Can also be "x", "xy", "xyz" for specific data
    n_proc=8) # number of parallel processes for various methods

# import data
juspyce_vol.fit()

# data is stored in dataframes X, Y and Z
# format is: n_functional x n_parcels
print(juspyce_vol.X.shape, juspyce_vol.Y.shape)
display(juspyce_vol.X)
display(juspyce_vol.Y)
display(juspyce_vol.Z)

INFO:juspyce.api:Checking input data for x (should be, e.g., PET data):
INFO:juspyce.io:Input type: list, assuming imaging data.


Parcellating imaging data (8 proc):   0%|          | 0/7 [00:00<?, ?it/s]

INFO:juspyce.api:Got 'x' data for 7 x 116 parcels.
INFO:juspyce.api:Checking input data for y (should be, e.g., subject data):
INFO:juspyce.io:Input type: list, assuming imaging data.


Parcellating imaging data (8 proc):   0%|          | 0/28 [00:00<?, ?it/s]

INFO:juspyce.api:Got 'y' data for 28 x 116 parcels.
INFO:juspyce.api:Checking input data for z (should be, e.g., grey matter data):
INFO:juspyce.api:Using nilearn grey matter template as 'z' to control for GMV.
INFO:juspyce.io:Input type: list, assuming imaging data.


...

INFO:juspyce.api:Got 'z' data for 1 x 116 parcels.
INFO:juspyce.api:Z-standardizing 'X' data.
INFO:juspyce.api:Z-standardizing 'Y' data.
INFO:juspyce.api:Z-standardizing 'Z' data.


(7, 116) (28, 116)


,LH_Vis_1,LH_Vis_2,LH_Vis_3,LH_Vis_4,LH_Vis_5,LH_Vis_6,LH_Vis_7,LH_Vis_8,LH_Vis_9,LH_SomMot_1,...,PUT-rh,CAU-rh,HIP-lh,AMY-lh,pTHA-lh,aTHA-lh,NAc-lh,GP-lh,PUT-lh,CAU-lh
5HT2a-cimbi36-29-beliveau2017,-0.264168,0.619941,0.514632,-0.052629,0.985569,0.767391,0.978715,0.106174,0.038989,1.037732,...,-1.799079,-3.109406,-0.785152,-1.336239,-2.100968,-1.413720,-1.041542,-2.344997,-1.331977,-3.089251
NMDA-ge179-29-galovic2021,-0.604138,0.479129,0.824289,-2.400195,-0.369305,0.695164,-0.704261,-1.197996,0.033065,0.199374,...,2.214798,1.118777,1.701154,-0.047469,2.937761,2.466495,1.534875,1.358547,2.503007,1.408804
mGluR5-abp688-73-smart2019,-0.571617,-1.216179,-0.616632,-2.584671,-1.578982,-0.101176,0.129241,-0.981888,0.072335,0.738736,...,0.397593,-0.165991,-0.781848,-1.092088,-1.722625,-2.301382,1.202235,-3.299477,0.203655,-0.209401
MU-carfentanil-204-kantonen2020,-0.495141,-1.307902,-1.166701,-1.755306,-1.864846,-1.615505,-0.569246,-1.362724,-1.574736,-0.524004,...,1.390534,1.645285,-0.818641,1.606248,1.635952,2.613713,3.758146,-0.246388,1.480701,1.533651
GABAa-flumazenil-6-dukart2018,0.608313,1.187470,1.537523,-1.190966,0.859510,1.948552,0.162192,-0.318240,0.649862,0.309374,...,-2.013617,-2.963909,-0.315344,-0.715506,-2.387318,-2.490360,-1.071716,-2.617411,-1.828161,-2.884014
5HT1b-p943-65-gallezot2010,-2.026874,-0.767668,-0.748779,-0.554213,1.202359,1.646692,-0.090497,0.063097,0.292534,-0.242754,...,0.338575,-1.582449,-2.147360,-0.637618,-1.833340,-1.754440,1.394318,3.675709,0.343322,-1.624356
D2-raclopride-156-malen2022,-0.267509,-0.181701,-0.242144,-0.590686,-0.247245,-0.255069,-0.284163,-0.387137,-0.176834,-0.227523,...,4.577519,3.674336,-0.167294,0.029536,0.096265,0.157489,3.628440,2.128651,4.752495,3.212779


,LH_Vis_1,LH_Vis_2,LH_Vis_3,LH_Vis_4,LH_Vis_5,LH_Vis_6,LH_Vis_7,LH_Vis_8,LH_Vis_9,LH_SomMot_1,...,PUT-rh,CAU-rh,HIP-lh,AMY-lh,pTHA-lh,aTHA-lh,NAc-lh,GP-lh,PUT-lh,CAU-lh
control,-0.979917,-0.384432,-0.724519,-0.717717,-0.847953,-0.845561,-1.027770,-0.109973,-0.372873,-0.860997,...,0.679512,-0.053890,-0.974177,-1.036892,1.822552,0.558228,-1.344703,0.795150,1.044851,-0.704277
touch,-0.609990,-0.623416,-0.441366,-0.674736,-0.539122,-0.611563,-0.615599,-0.537207,-0.602477,1.396591,...,0.071053,-0.609950,-0.748519,-0.735118,0.548597,-0.081033,-0.698296,-0.690818,-0.446518,-0.728646
interoception,-0.584510,-0.309503,-0.441553,-0.536978,-0.496687,-0.203884,-0.614538,-0.720578,-0.650276,1.074812,...,3.057332,0.294060,-0.439937,0.529685,0.217025,0.013952,-0.016668,0.331524,0.703412,-0.101138
learning,1.881603,0.282980,0.049563,0.306161,0.083300,0.035737,-0.237220,0.198877,-0.146356,-1.038012,...,1.827821,1.621217,2.805222,0.205436,0.481021,0.841901,3.329236,3.235139,2.791219,1.956812
attention,-1.043345,0.988413,-0.013416,1.415960,1.346545,0.263423,1.086665,1.675413,0.603983,-0.583783,...,-0.431557,-0.733922,-1.437557,-1.884704,-0.361686,-0.513677,-1.552835,-0.952838,-1.012371,-0.954968
language,-0.361627,-0.218567,-0.596183,0.146000,-0.766066,-0.385798,0.425113,-0.574921,-0.527282,0.782032,...,-0.106558,-0.043684,-0.791802,-0.940110,-0.429036,0.198915,-0.120527,0.305078,-0.036820,-0.008570
interaction,0.562361,0.850952,-0.019251,1.163435,1.143413,-0.104980,0.415113,0.595721,-0.172755,2.326707,...,0.307117,-0.440936,0.887388,-0.834922,0.602970,-0.470414,-1.045377,0.412924,0.401740,-1.089458
inhibition,-1.229373,-0.802153,-0.687448,-0.683081,-0.928564,-0.953441,-0.938040,-0.531735,-0.650699,-0.140440,...,0.623672,0.052508,-1.389163,-1.310390,0.860164,-0.109300,-1.087186,0.796897,1.026083,-0.615228
somatosensory,-0.621487,-0.691599,-0.454639,-0.685211,-0.589551,-0.574190,-0.680714,-0.607947,-0.639705,1.438160,...,0.043530,-0.755064,-0.771641,-0.772320,0.287000,-0.240668,-0.637103,-0.803458,-0.458228,-0.727205
decision,-0.925355,0.197618,-0.621515,0.203955,-0.430108,-0.760909,0.794073,-0.055139,-0.471957,-1.359569,...,0.855850,1.791906,-1.361423,-1.065043,-0.167914,0.885781,4.084600,3.309685,2.278896,2.176497


,LH_Vis_1,LH_Vis_2,LH_Vis_3,LH_Vis_4,LH_Vis_5,LH_Vis_6,LH_Vis_7,LH_Vis_8,LH_Vis_9,LH_SomMot_1,...,PUT-rh,CAU-rh,HIP-lh,AMY-lh,pTHA-lh,aTHA-lh,NAc-lh,GP-lh,PUT-lh,CAU-lh
gm,0.921333,0.17635,0.697561,-1.638616,-0.881545,0.259694,-0.339806,-1.042037,-0.933184,-0.88505,...,-0.075629,1.104424,1.833296,2.346207,0.271519,0.381889,2.348057,-3.565602,-0.349896,1.576679


### Save juspyce object and reload

In [4]:
# juspyce objects can be saved any time including all data. 
# compression is defined by ending: either *.pkl/*.pickle (no compression) 
# or *.pkl.gz/*.pickle.gz (gzip compression)
juspyce_vol.to_pickle(os.path.join(wd, "testing", "test_juspyce_vol.pkl.gz")) 
juspyce_vol = JuSpyce.from_pickle(os.path.join(wd, "testing", "test_juspyce_vol.pkl.gz"))

INFO:juspyce.api:Saved complete gzip compressed object to /Users/llotter/projects/juspyce/testing/test_juspyce_vol.pkl.gz.
INFO:juspyce.api:Loaded complete object from /Users/llotter/projects/juspyce/testing/test_juspyce_vol.pkl.gz.


## Initialize & import: mixed volume and surface data

In [5]:
# initialize
# volumetric data can be transformed to surface. CAVE: not the other way round!
juspyce_surf = JuSpyce(
    x=predictors_surf, # list of input data to predict functional data from -> PET surfaces
    y=functional_vol, # list of functional data -> volumetric, will be transformed to surface
    z="gm", # will be transformed to surface
    data_space=["fsaverage", "MNI152", "MNI152"], # data spaces: x -> surface, y & z-> volumetric 
    parcellation=parcellation_surf, # surface parcellation
    parcellation_labels=None, # if no parcellation labels given, will create numbered columns
    parcellation_space="fsaverage", # surface space
    standardize=False, 
    n_proc=8) 

# import data
juspyce_surf.fit()

# data is stored in dataframes X, Y and Z
# format is: n_functional x n_parcels
print(juspyce_surf.X.shape, juspyce_surf.Y.shape)
display(juspyce_surf.X)
display(juspyce_surf.Y)
display(juspyce_surf.Z)

INFO:juspyce.api:Checking input data for x (should be, e.g., PET data):
INFO:juspyce.io:Input type: list, assuming imaging data.


Parcellating imaging data (8 proc):   0%|          | 0/2 [00:00<?, ?it/s]

INFO:juspyce.api:Got 'x' data for 2 x 148 parcels.
INFO:juspyce.api:Checking input data for y (should be, e.g., subject data):
INFO:juspyce.io:Input type: list, assuming imaging data.


Parcellating imaging data (8 proc):   0%|          | 0/28 [00:00<?, ?it/s]

INFO:juspyce.api:Got 'y' data for 28 x 148 parcels.
INFO:juspyce.api:Checking input data for z (should be, e.g., grey matter data):
INFO:juspyce.api:Using nilearn grey matter template as 'z' to control for GMV.
INFO:juspyce.io:Input type: list, assuming imaging data.


...

INFO:juspyce.api:Got 'z' data for 1 x 148 parcels.


(2, 148) (28, 148)


,0,1,2,3,4,5,6,7,8,9,...,138,139,140,141,142,143,144,145,146,147
CMRo2-o2-33-Vaishnavi2010_L.func,4960.856445,4723.979492,4764.959961,4568.198730,5060.350098,4829.768066,4780.229492,4694.488770,4928.132812,4676.219238,...,4925.309570,4742.232910,4903.771973,4964.388672,4669.261230,4881.380859,4867.863770,4995.543457,4785.766602,5118.133301
CBF-h2o-33-Vaishnavi2010_L.func,5764.210938,5409.136719,5516.737793,5269.058105,5779.020508,5583.581055,5569.392090,5428.163086,5664.505859,5352.337402,...,5744.316895,5515.418457,5694.718262,5750.099121,5368.715332,5688.740723,5705.983887,5787.997559,5547.638672,5939.518066


,0,1,2,3,4,5,6,7,8,9,...,138,139,140,141,142,143,144,145,146,147
control,-1.371014,-1.346159,1.016378,0.448044,-1.255325,-2.024646,2.398735,2.690309,-1.071807,-0.696488,...,-0.043549,-0.670861,2.472193,0.420016,3.870055,-3.325713,-0.631142,-1.161160,-2.627733,-3.380851
touch,-1.289809,-1.366971,2.814862,6.081244,-0.805127,-1.483805,-1.036637,1.155758,-0.751748,-0.767616,...,-1.237522,0.834570,7.807311,-0.749544,0.205293,-1.554056,-0.889442,-0.701713,-0.399285,2.733511
interoception,-1.179117,-1.140074,-0.930808,2.543342,-0.659694,0.102520,0.434319,-0.598213,-1.154665,-0.736761,...,-1.183257,0.451261,-0.071721,0.232134,-1.342661,-1.040956,-1.023058,-0.778582,-0.537093,1.277936
learning,-1.572913,1.269572,-1.654516,-0.961098,0.006065,-1.403230,-0.692625,-0.414042,-0.568809,0.352733,...,-0.301428,-0.498740,-1.212603,-1.474852,0.113258,-1.508902,-1.382449,-0.901932,-2.255096,-3.998271
attention,-1.579017,4.848795,-2.015037,-1.845505,-1.452127,-3.097129,0.807872,-0.558523,-0.706361,-0.419569,...,2.964164,-1.198841,1.652925,2.049416,3.704588,-3.627681,-0.217847,-0.423442,-0.353715,-1.395062
language,-1.026605,0.524173,-0.905539,0.478666,-0.888232,-1.450918,1.486510,-0.772841,-0.566288,-0.863235,...,-0.586411,-0.920050,-1.329494,-0.111244,-0.622946,-1.437327,-0.963530,0.184522,0.825368,-0.103948
interaction,-2.641033,3.546570,-0.186795,2.443621,-1.907089,-3.282254,-0.955571,-0.092537,-2.625381,-0.742452,...,-0.544461,-1.711683,1.155326,-0.010440,1.990413,-3.530236,-2.145781,-0.069053,2.018002,4.889253
inhibition,-1.956237,-1.963915,3.559033,3.063316,-1.684497,-2.576224,3.008456,4.504025,-1.551673,-1.398739,...,-1.199847,-0.475013,5.500950,1.882685,5.664635,-3.518324,-0.897055,-1.324444,-2.064466,-2.005240
somatosensory,-1.033058,-1.414063,2.547990,5.947804,-0.529854,-1.177540,-0.893077,1.073547,-0.500831,-0.667135,...,-1.110196,1.000544,7.026681,-0.700827,0.284644,-1.285796,-0.792750,-0.646177,-0.119571,2.373124
decision,-0.858616,2.153281,-1.658931,-1.498151,-0.805422,-0.384554,2.954711,-0.227272,-0.712050,-1.225212,...,-0.628846,-0.548395,-1.298228,0.823443,1.251700,-0.587261,-1.332829,-0.684007,-1.424613,-4.121480


,0,1,2,3,4,5,6,7,8,9,...,138,139,140,141,142,143,144,145,146,147
gm,0.552675,0.580925,0.554485,0.77889,0.560578,0.864765,0.843337,0.829759,0.922103,0.76727,...,0.743937,0.553545,0.625658,0.735472,0.546614,0.828356,0.807441,0.69386,0.59458,0.756037


### Initialize & import: numpy arrays

In [6]:
# we use the arrays from the parcellated volume data
# the column & index labels will get lost on the way

x = juspyce_vol.X.iloc[0,:].values # 1d array
y = juspyce_vol.Y.values # 2d array
print(type(x), x.shape)
print(type(y), y.shape)

juspyce_numpy = JuSpyce(x=x, y=y).fit()
display(juspyce_numpy.X)
display(juspyce_numpy.Y)

INFO:juspyce.api:Checking input data for x (should be, e.g., PET data):
INFO:juspyce.io:Input type: ndarray, assuming parcellated data with shape (n_files/subjects/etc, n_parcels).
INFO:juspyce.api:Got 'x' data for 1 x 116 parcels.
INFO:juspyce.api:Checking input data for y (should be, e.g., subject data):
INFO:juspyce.io:Input type: ndarray, assuming parcellated data with shape (n_files/subjects/etc, n_parcels).
INFO:juspyce.api:Got 'y' data for 28 x 116 parcels.
INFO:juspyce.api:Z-standardizing 'X' data.
INFO:juspyce.api:Z-standardizing 'Y' data.


<class 'numpy.ndarray'> (116,)
<class 'numpy.ndarray'> (28, 116)


,0,1,2,3,4,5,6,7,8,9,...,106,107,108,109,110,111,112,113,114,115
0,-0.264167,0.619942,0.514633,-0.052628,0.985569,0.767391,0.978715,0.106174,0.038989,1.037732,...,-1.799079,-3.109406,-0.785151,-1.336239,-2.100968,-1.41372,-1.041542,-2.344997,-1.331977,-3.089251


,0,1,2,3,4,5,6,7,8,9,...,106,107,108,109,110,111,112,113,114,115
0,-0.979917,-0.384432,-0.724519,-0.717717,-0.847953,-0.845560,-1.027770,-0.109973,-0.372873,-0.860997,...,0.679512,-0.053890,-0.974176,-1.036891,1.822552,0.558228,-1.344703,0.795150,1.044851,-0.704277
1,-0.609990,-0.623416,-0.441366,-0.674736,-0.539122,-0.611564,-0.615599,-0.537207,-0.602477,1.396591,...,0.071053,-0.609950,-0.748519,-0.735118,0.548597,-0.081033,-0.698296,-0.690819,-0.446518,-0.728646
2,-0.584510,-0.309503,-0.441553,-0.536978,-0.496687,-0.203884,-0.614538,-0.720578,-0.650276,1.074812,...,3.057332,0.294060,-0.439937,0.529685,0.217025,0.013952,-0.016668,0.331524,0.703412,-0.101138
3,1.881603,0.282980,0.049563,0.306161,0.083300,0.035737,-0.237220,0.198877,-0.146356,-1.038011,...,1.827821,1.621217,2.805222,0.205435,0.481021,0.841900,3.329236,3.235139,2.791219,1.956812
4,-1.043345,0.988413,-0.013416,1.415961,1.346545,0.263423,1.086665,1.675413,0.603983,-0.583783,...,-0.431557,-0.733922,-1.437557,-1.884704,-0.361686,-0.513677,-1.552835,-0.952838,-1.012371,-0.954968
5,-0.361627,-0.218567,-0.596183,0.146000,-0.766066,-0.385798,0.425112,-0.574921,-0.527282,0.782032,...,-0.106558,-0.043684,-0.791802,-0.940110,-0.429037,0.198915,-0.120527,0.305078,-0.036820,-0.008570
6,0.562361,0.850952,-0.019251,1.163435,1.143413,-0.104980,0.415113,0.595721,-0.172755,2.326707,...,0.307118,-0.440936,0.887388,-0.834922,0.602970,-0.470414,-1.045377,0.412925,0.401740,-1.089459
7,-1.229373,-0.802153,-0.687448,-0.683081,-0.928564,-0.953441,-0.938040,-0.531735,-0.650699,-0.140440,...,0.623672,0.052508,-1.389163,-1.310391,0.860165,-0.109300,-1.087187,0.796897,1.026083,-0.615229
8,-0.621487,-0.691599,-0.454639,-0.685210,-0.589551,-0.574190,-0.680714,-0.607947,-0.639705,1.438160,...,0.043530,-0.755064,-0.771641,-0.772320,0.287000,-0.240668,-0.637103,-0.803458,-0.458228,-0.727205
9,-0.925355,0.197618,-0.621516,0.203956,-0.430108,-0.760909,0.794073,-0.055139,-0.471957,-1.359570,...,0.855850,1.791906,-1.361423,-1.065043,-0.167914,0.885781,4.084601,3.309685,2.278896,2.176497


### Initialize & import: pandas df/series

In [7]:
# we use the arrays from the parcellated volume data
# the column & index labels will get lost on the way

x = juspyce_vol.X.iloc[0,:] # series
y = juspyce_vol.Y # dataframe
print(type(x), x.shape)
print(type(y), y.shape)

juspyce_pandas = JuSpyce(x=x, y=y).fit()
display(juspyce_pandas.X)
display(juspyce_pandas.Y)

INFO:juspyce.api:Checking input data for x (should be, e.g., PET data):
INFO:juspyce.io:Input type: Series, assuming parcellated data with shape (1, n_parcels).
INFO:juspyce.api:Got 'x' data for 1 x 116 parcels.
INFO:juspyce.api:Checking input data for y (should be, e.g., subject data):
INFO:juspyce.io:Input type: DataFrame, assuming parcellated data with shape (n_files/subjects/etc, n_parcels).
INFO:juspyce.api:Got 'y' data for 28 x 116 parcels.
INFO:juspyce.api:Z-standardizing 'X' data.
INFO:juspyce.api:Z-standardizing 'Y' data.


<class 'pandas.core.series.Series'> (116,)
<class 'pandas.core.frame.DataFrame'> (28, 116)


,LH_Vis_1,LH_Vis_2,LH_Vis_3,LH_Vis_4,LH_Vis_5,LH_Vis_6,LH_Vis_7,LH_Vis_8,LH_Vis_9,LH_SomMot_1,...,PUT-rh,CAU-rh,HIP-lh,AMY-lh,pTHA-lh,aTHA-lh,NAc-lh,GP-lh,PUT-lh,CAU-lh
5HT2a-cimbi36-29-beliveau2017,-0.264167,0.619942,0.514633,-0.052628,0.985569,0.767391,0.978715,0.106174,0.038989,1.037732,...,-1.799079,-3.109406,-0.785151,-1.336239,-2.100968,-1.41372,-1.041542,-2.344997,-1.331977,-3.089251


,LH_Vis_1,LH_Vis_2,LH_Vis_3,LH_Vis_4,LH_Vis_5,LH_Vis_6,LH_Vis_7,LH_Vis_8,LH_Vis_9,LH_SomMot_1,...,PUT-rh,CAU-rh,HIP-lh,AMY-lh,pTHA-lh,aTHA-lh,NAc-lh,GP-lh,PUT-lh,CAU-lh
control,-0.979917,-0.384432,-0.724519,-0.717717,-0.847953,-0.845560,-1.027770,-0.109973,-0.372873,-0.860997,...,0.679512,-0.053890,-0.974176,-1.036891,1.822552,0.558228,-1.344703,0.795150,1.044851,-0.704277
touch,-0.609990,-0.623416,-0.441366,-0.674736,-0.539122,-0.611564,-0.615599,-0.537207,-0.602477,1.396591,...,0.071053,-0.609950,-0.748519,-0.735118,0.548597,-0.081033,-0.698296,-0.690819,-0.446518,-0.728646
interoception,-0.584510,-0.309503,-0.441553,-0.536978,-0.496687,-0.203884,-0.614538,-0.720578,-0.650276,1.074812,...,3.057332,0.294060,-0.439937,0.529685,0.217025,0.013952,-0.016668,0.331524,0.703412,-0.101138
learning,1.881603,0.282980,0.049563,0.306161,0.083300,0.035737,-0.237220,0.198877,-0.146356,-1.038011,...,1.827821,1.621217,2.805222,0.205435,0.481021,0.841900,3.329236,3.235139,2.791219,1.956812
attention,-1.043345,0.988413,-0.013416,1.415961,1.346545,0.263423,1.086665,1.675413,0.603983,-0.583783,...,-0.431557,-0.733922,-1.437557,-1.884704,-0.361686,-0.513677,-1.552835,-0.952838,-1.012371,-0.954968
language,-0.361627,-0.218567,-0.596183,0.146000,-0.766066,-0.385798,0.425112,-0.574921,-0.527282,0.782032,...,-0.106558,-0.043684,-0.791802,-0.940110,-0.429037,0.198915,-0.120527,0.305078,-0.036820,-0.008570
interaction,0.562361,0.850952,-0.019251,1.163435,1.143413,-0.104980,0.415113,0.595721,-0.172755,2.326707,...,0.307118,-0.440936,0.887388,-0.834922,0.602970,-0.470414,-1.045377,0.412925,0.401740,-1.089459
inhibition,-1.229373,-0.802153,-0.687448,-0.683081,-0.928564,-0.953441,-0.938040,-0.531735,-0.650699,-0.140440,...,0.623672,0.052508,-1.389163,-1.310391,0.860165,-0.109300,-1.087187,0.796897,1.026083,-0.615229
somatosensory,-0.621487,-0.691599,-0.454639,-0.685210,-0.589551,-0.574190,-0.680714,-0.607947,-0.639705,1.438160,...,0.043530,-0.755064,-0.771641,-0.772320,0.287000,-0.240668,-0.637103,-0.803458,-0.458228,-0.727205
decision,-0.925355,0.197618,-0.621516,0.203956,-0.430108,-0.760909,0.794073,-0.055139,-0.471957,-1.359570,...,0.855850,1.791906,-1.361423,-1.065043,-0.167914,0.885781,4.084601,3.309685,2.278896,2.176497
